In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.applications.xception import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import SGD
import numpy as np
import pickle as pkl
import datetime
from skimage import exposure

# Load Cifar10 dataset

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [4]:
print('The training set has {0} samples with dimentions {1}.'.format(x_train.shape[0], x_train.shape[1:]))
print('The test set has {0} samples with dimentions {1}.'.format(x_test.shape[0], x_test.shape[1:]))

The training set has 50000 samples with dimentions (32, 32, 3).
The test set has 10000 samples with dimentions (32, 32, 3).


In [5]:
NUM_CLASSES = 10
IMG_ROWS, IMG_COLS = 32, 32
BATCH_SIZE = 512
EPOCHS = 20
VAL_SPLIT = .2
LEARNING_RATE = 1e-2
SGD_MOMENTUM = .9

# Preprocess data

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [7]:
x_train -= np.mean(x_train)
x_test -= np.mean(x_test)

In [8]:
x_train /= np.std(x_train)
x_test /= np.std(x_test)

In [9]:
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [10]:
validation_split = int((1-VAL_SPLIT)*len(x_train))
x_train, x_val = x_train[:validation_split], x_train[validation_split:]
y_train, y_val = y_train[:validation_split], y_train[validation_split:]

In [11]:
def contrast_stretching(img):
    p2, p98 = np.percentile(img, (2, 98))
    return exposure.rescale_intensity(img, in_range=(p2, p98))

In [12]:
datagen = ImageDataGenerator(preprocessing_function=contrast_stretching)

# Build model

In [13]:
input_tensor = Input(shape=(IMG_ROWS, IMG_COLS, 3))

In [14]:
base_model = Xception(include_top=False, input_tensor=input_tensor)

In [16]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [17]:
xception = Model(inputs=base_model.input, outputs=predictions)

In [18]:
xception.compile(optimizer=SGD(lr=LEARNING_RATE, momentum=SGD_MOMENTUM), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model

In [19]:
now = datetime.datetime.now()
tensorflow_log_dir = "Graph/" + now.strftime("%Y%m%d-%H%M%S") + "/"

In [20]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorflow_log_dir, histogram_freq=0,  
          write_graph=False, write_images=False)

In [21]:
xception.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(x_train) / BATCH_SIZE, 
                    validation_data=ImageDataGenerator().flow(x_val, y_val, batch_size=BATCH_SIZE),
                    validation_steps=len(x_val) / BATCH_SIZE,
                    epochs=EPOCHS, callbacks=[tensorboard_callback], verbose=1)

Epoch 1/100
79/78 [==============================] - 80s 1s/step - loss: 2.0397 - acc: 0.2715 - val_loss: 1.8602 - val_acc: 0.3744
Epoch 2/100
79/78 [==============================] - 59s 748ms/step - loss: 1.6514 - acc: 0.4541 - val_loss: 1.5482 - val_acc: 0.4792
Epoch 3/100
79/78 [==============================] - 60s 762ms/step - loss: 1.3034 - acc: 0.5455 - val_loss: 1.2660 - val_acc: 0.5568
Epoch 4/100
79/78 [==============================] - 61s 769ms/step - loss: 1.1408 - acc: 0.5992 - val_loss: 1.1612 - val_acc: 0.5960
Epoch 5/100
79/78 [==============================] - 61s 766ms/step - loss: 1.0467 - acc: 0.6310 - val_loss: 1.0876 - val_acc: 0.6199
Epoch 6/100
79/78 [==============================] - 61s 766ms/step - loss: 0.9864 - acc: 0.6531 - val_loss: 1.0227 - val_acc: 0.6403
Epoch 7/100
79/78 [==============================] - 60s 762ms/step - loss: 0.9392 - acc: 0.6692 - val_loss: 0.9933 - val_acc: 0.6447
Epoch 8/100
79/78 [==============================] - 60s 758ms/st

Epoch 62/100
79/78 [==============================] - 60s 763ms/step - loss: 0.2945 - acc: 0.8966 - val_loss: 0.7642 - val_acc: 0.7712
Epoch 63/100
79/78 [==============================] - 60s 765ms/step - loss: 0.2950 - acc: 0.8957 - val_loss: 0.8028 - val_acc: 0.7628
Epoch 64/100
79/78 [==============================] - 60s 759ms/step - loss: 0.2910 - acc: 0.8980 - val_loss: 0.7621 - val_acc: 0.7759
Epoch 65/100
79/78 [==============================] - 60s 760ms/step - loss: 0.2831 - acc: 0.9009 - val_loss: 0.7593 - val_acc: 0.7769
Epoch 66/100
79/78 [==============================] - 60s 765ms/step - loss: 0.2771 - acc: 0.9021 - val_loss: 0.7745 - val_acc: 0.7795
Epoch 67/100
79/78 [==============================] - 60s 763ms/step - loss: 0.2750 - acc: 0.9030 - val_loss: 0.7859 - val_acc: 0.7800
Epoch 68/100
79/78 [==============================] - 60s 765ms/step - loss: 0.2665 - acc: 0.9059 - val_loss: 0.7601 - val_acc: 0.7806
Epoch 69/100
79/78 [==============================] - 6

In [22]:
xception.evaluate(x_test, y_test, verbose=1)

10000/10000 [==============================] - 9s 930us/step


[2.101669556045532, 0.543]